In [1]:
with open("../config/app_id.txt") as f:
    app_id = f.read()

In [21]:
from bs4 import BeautifulSoup
import requests as req
from tqdm import tqdm
import pandas as pd
import numpy as np
import json


In [22]:
link = f"https://api.worldoftanks.ru/wot/encyclopedia/vehicles/?application_id={app_id}&limit=1"


In [23]:
data = json.loads(req.get(link).text)


In [161]:
tanks_data = []
tanks_index = []
try:
    for tank_id in tqdm(data["data"]):
        #tank_id = "13889" 

        tank = {}
        tank["wg_id"] = tank_id
        tank["tag"] = data["data"][tank_id]["tag"]
        tank["lvl"] = data["data"][tank_id]["tier"]
        tank["type"] = data["data"][tank_id]["type"]
        tank["name"] = data["data"][tank_id]["name"]
        tank["short_name"] = data["data"][tank_id]["short_name"]

        modules = {}
        modules["vehicleTurret"] = []
        modules["vehicleGun"] = []
        for i in data["data"][tank_id]["modules_tree"]:
            if data["data"][tank_id]["modules_tree"][i]["type"] == "vehicleTurret":
                modules["vehicleTurret"].append(i)
            elif data["data"][tank_id]["modules_tree"][i]["type"] == "vehicleGun":
                    modules["vehicleGun"].append(i)
            elif data["data"][tank_id]["modules_tree"][i]["next_modules"] is None:
                    modules[data["data"][tank_id]["modules_tree"][i]["type"]] = i

        if len(modules["vehicleTurret"]) > 1:
            if data["data"][tank_id]["modules_tree"][modules["vehicleTurret"][0]]['is_default']:
                modules["vehicleTurret"] = modules["vehicleTurret"][1]
            else:
                modules["vehicleTurret"] = modules["vehicleTurret"][0]
        elif len(modules["vehicleTurret"]) == 0:
            modules["vehicleTurret"] = None
        else:
            modules["vehicleTurret"] = modules["vehicleTurret"][0]

        config = {}
        dpm = 0

        for i in modules["vehicleGun"]:
            link = f"https://api.worldoftanks.ru/wot/encyclopedia/vehicleprofile/?application_id={app_id}"
            link += "&tank_id=" + tank_id
            link += "&gun_id=" + i
            link += "&suspension_id=" + modules["vehicleChassis"]
            link += "&engine_id=" + modules["vehicleEngine"]
            link += "&radio_id=" + modules["vehicleRadio"]
            if modules["vehicleTurret"] is not None:
                link += "&turret_id=" + modules["vehicleTurret"]
            tmp_config = json.loads(req.get(link).text)
            
            if tmp_config["status"] == "error":
                continue
            tmp_dpm = tmp_config["data"][tank_id]["ammo"][0]["damage"][1] * tmp_config["data"][tank_id]["gun"]["fire_rate"]
            
            if dpm < tmp_dpm:
                dpm = tmp_dpm
                config = tmp_config

        tank["damage"] = config["data"][tank_id]["ammo"][0]["damage"][1]
        tank["dpm"] = round(dpm, 2)
        tank["penetration"] = config["data"][tank_id]["ammo"][0]["penetration"][1]
        tank["accuracy"] = config["data"][tank_id]["gun"]["dispersion"]
        tank["aim_time"] = config["data"][tank_id]["gun"]["aim_time"]
        tank["speed"] = config["data"][tank_id]["speed_forward"]
        tank["cp_power"] = round(config["data"][tank_id]["engine"]["power"] / (config["data"][tank_id]["weight"] / 1000), 2)
        tank["hp"] = config["data"][tank_id]["hp"]
        tank["armor_hull"] = config["data"][tank_id]["armor"]["hull"]["front"]
        if config["data"][tank_id]["armor"]["turret"] is not None:
            tank["armor_turret"] = config["data"][tank_id]["armor"]["turret"]["front"]
            tank["view_range"] = config["data"][tank_id]["turret"]["view_range"]
        else:
            tank["armor_turret"] = np.nan
        tank["view_range"] = config["data"][tank_id]["turret"]["view_range"]

        tanks_data.append(tank)
        tanks_index.append(tank_id)
except Exception as e:
    print(tank_id)
    print(str(e))
    print(modules)
    print(config)
    
tanks_data = pd.DataFrame(tanks_data, index=tanks_index)


100%|██████████| 713/713 [05:13<00:00,  2.27it/s]


In [162]:
# Remove special mods tanks
tanks_data = tanks_data[~tanks_data.name.map(lambda x: any([x.endswith(i) for i in ["ЛФ", "FL", "CL", "КЛ"]]))]


In [163]:
scripts = BeautifulSoup(req.get("http://wotreplays.ru").text, "html.parser").findAll(lambda tag: tag.name == "script" and tag.has_attr("src"))
for i in scripts:
    if i["src"].endswith("ru.filters.js"):
        link = "http://wotreplays.ru" + i["src"]
        break

js_data = req.get(link).text
wr_tanks_json = json.loads(js_data[len("var filtersData = "):js_data.find(";")])
wr_tanks = pd.DataFrame(wr_tanks_json["tank"]["checkboxes"])[["id", "title"]]\
    .rename(columns={"id": "wr_id", "title": "name"})\
    .replace("Progetto M35 mod 46", "Progetto M35 mod. 46")\
    .replace("M4 Sherman", "M4A1 Sherman")\
    .replace("Carro da Combattimento 45t", "Carro da Combattimento 45 t")\
    .replace("CS-52", "CS-52 LIS")


In [164]:
# Checking for tanks we didn't get wr_id
md = tanks_data.merge(wr_tanks, on="name", how="left")
md[md.wr_id.isnull()]

,wg_id,tag,lvl,type,name,short_name,damage,dpm,penetration,accuracy,aim_time,speed,cp_power,hp,armor_hull,armor_turret,view_range,wr_id


In [165]:
md

,wg_id,tag,lvl,type,name,short_name,damage,dpm,penetration,accuracy,aim_time,speed,cp_power,hp,armor_hull,armor_turret,view_range,wr_id
0,1,R04_T-34,5,mediumTank,Т-34,Т-34,85,2217.65,112,0.34,2.3,56,17.40,590,45,52.0,350,467-795
1,3137,F08_AMX_50_100,8,heavyTank,AMX 50 100,AMX 50 100,240,1776.00,212,0.36,3.0,51,17.11,1400,90,90.0,380,378-858
2,2049,R12_A-20,5,lightTank,А-20,А-20,40,2168.80,46,0.45,2.3,72,28.39,560,20,37.0,360,486-1004
3,15617,R95_Object_907,10,mediumTank,Объект 907,Об. 907,320,2953.60,264,0.36,2.0,55,16.94,2000,110,228.0,400,669-1057
4,49169,G136_Tiger_131,6,heavyTank,Tiger 131,Tiger 131,220,1885.40,145,0.38,2.3,40,11.40,1060,100,100.0,370,1280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,545,A01_T1_Cunningham,1,lightTank,T1 Cunningham,T1,30,900.00,33,0.51,2.5,41,17.47,245,10,10.0,280,1033
715,4705,J21_Type_91,3,heavyTank,Type 91 Heavy,Type 91,70,1999.90,81,0.40,2.1,25,12.44,445,20,20.0,330,892
716,47873,R143_T_29,3,mediumTank,Т-29,Т-29,110,1410.20,66,0.55,2.3,45,17.54,445,30,20.0,300,1281
717,1841,Ch18_WZ-120,9,mediumTank,WZ-120,WZ-120,440,2398.00,249,0.38,3.4,60,16.13,1750,100,220.0,390,617


In [167]:
md.to_csv("../data/assets/tanks.csv", index=False)
